## `map_region_city_coordinates_v2.py`
1. Read the csv file from the `Output/output03-gnoc_networks-countrycodes.csv` into pandas dataframe
2. Populate new columns  in dataframe by performing jmespath search in `CityDatabase/regions.json` 
3. Write file to `Output/output04-gnoc_networks-region_city_coordinates.csv`and dropped columns to another csv files.

In [1]:
import pandas as pd
import json
import jmespath
import numpy as np
import os.path
import time

In [2]:
df = pd.read_csv("Output/output03-gnoc_networks-countrycodes.csv", encoding='utf8', sep=';')

In [3]:
df.sort_values(by=['Country'],inplace=True)
df.head()
df2 = df.copy()
df2.head()

,Network,City,Country,CountryCode
2579,10.129.9.192/26,Buenos Aires,Argentina,AR
7045,10.129.13.0/24,Buenos Aires,Argentina,AR
7046,10.129.3.0/24,Buenos Aires,Argentina,AR
7047,10.129.2.0/24,Buenos Aires,Argentina,AR
7048,10.129.7.0/27,Buenos Aires,Argentina,AR


In [4]:
df.to_csv("Output/output05-gnoc_networks-region_city_coordinates-sorted.csv.csv", encoding="utf8", index=False)


In [5]:
# Populate Lookup key

df['LookupKey'] = df['City'] + str('#') + df['Country'] + str('#') + df['CountryCode']
df.head()


,Network,City,Country,CountryCode,LookupKey
2579,10.129.9.192/26,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR
7045,10.129.13.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR
7046,10.129.3.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR
7047,10.129.2.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR
7048,10.129.7.0/27,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR


In [6]:
df2.drop_duplicates(subset=['City','Country','CountryCode'], inplace=True)
df2.drop(columns=['Network'], inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283 entries, 2579 to 10200
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   City         283 non-null    object
 1   Country      283 non-null    object
 2   CountryCode  283 non-null    object
dtypes: object(3)
memory usage: 8.8+ KB


In [7]:
df2.head()

,City,Country,CountryCode
2579,Buenos Aires,Argentina,AR
10188,FW-unknown,Argentina,AR
5355,Adelaide,Australia,AU
5318,Sydney,Australia,AU
11083,Melbourne,Australia,AU


In [8]:
def search_latitude(x):
    filename = "CityDatabase/Cities/"+str(x['CountryCode'])+".json"

    if(os.path.exists(filename)==True):
        regions = open(filename, mode='r',encoding ="utf8")
        data = json.load(regions, encoding="utf8")

        latitude_query  = """regions[*].cities[?name=='"""+str(x['City'])+"""'].latitude"""
        
        path = jmespath.search(latitude_query,data)
        path = [y for y in path if y != []]
        
        if(len(path)==0): # City is not present in JSON file, hence return null
            #print("City not present in list")
            return(str(""))
        elif(len(path)==1): # Exactly 1 match is found, hence return latitude
            #print("Returning latitude")
            return path[0][0]
        elif(len(path)>1): # More than 1 city found, hence ambiguity about the region and lat-long
            #print("I am not that smart")
            return(str(""))
        data =""
        regions.close()
    else:
        return(str(np.nan))
    
start = time.time()
df2['Latitude'] = df2[['CountryCode','City']].apply(search_latitude,axis=1) 
end = time.time()

print(end - start)

# 678 seconds for 14k entries
# sorting the data by city: 382 sec

19.235795736312866


In [9]:
df2.head()

,City,Country,CountryCode,Latitude
2579,Buenos Aires,Argentina,AR,-34.6021
10188,FW-unknown,Argentina,AR,
5355,Adelaide,Australia,AU,-34.8716
5318,Sydney,Australia,AU,-33.8591
11083,Melbourne,Australia,AU,-37.81


In [10]:
def search_longitude(x):
    filename = "CityDatabase/Cities/"+str(x['CountryCode'])+".json"

    if(os.path.exists(filename)==True):
        regions = open(filename, mode='r',encoding ="utf8")
        data = json.load(regions, encoding="utf8")

        longitude_query  = """regions[*].cities[?name=='"""+str(x['City'])+"""'].longitude"""
        
        path = jmespath.search(longitude_query,data)
        path = [y for y in path if y != []]
        
        if(len(path)==0): # City is not present in JSON file, hence return null
            #print("City not present in list")
            return(str(""))
        elif(len(path)==1): # Exactly 1 match is found, hence return longitude
            #print("Returning latitude")
            return path[0][0]
        elif(len(path)>1): # More than 1 city found, hence ambiguity about the region and lat-long
            #print("I am not that smart")
            return(str(""))
        data =""
        regions.close()
    else:
        return(str(np.nan))
    
start = time.time()
df2['Longitude'] = df2[['CountryCode','City']].apply(search_longitude,axis=1)
end = time.time()
print(end - start)

# 684 seconds for 14k entries
# sorting helps 386 sec

19.120134115219116


In [11]:
df2.head()

,City,Country,CountryCode,Latitude,Longitude
2579,Buenos Aires,Argentina,AR,-34.6021,-58.3845
10188,FW-unknown,Argentina,AR,,
5355,Adelaide,Australia,AU,-34.8716,138.676
5318,Sydney,Australia,AU,-33.8591,151.2002
11083,Melbourne,Australia,AU,-37.81,144.9644


In [12]:
def search_region(x):
    
    filename = "CityDatabase/Cities/"+str(x['CountryCode'])+".json"

    if(os.path.exists(filename)==True):
        regions = open(filename, mode='r',encoding ="utf8")
        data = json.load(regions, encoding="utf8")

        region_query = """regions[].{region_name: name, city_names: cities[?name=='"""+str(x['City'])+"""'].name[]}"""

        path = jmespath.search(region_query,data)
        q = [x['region_name'] for x in path if (x['city_names'])]   # frigging list comprehension
        
        if(len(q)==0): # Region is not found in JSON file, hence return null
            #print("Region is not present in list")
            return(str(""))
        
        elif(len(q)==1): # Exactly 1 match is found, hence return region
            #print("Returning latitude")
            return str(q[0])
        
        elif(len(q)>1): # More than 1 regions found, hence ambiguity about the region
            #print("I am not that smart")
            return(str(""))
        data =""
        regions.close()
        
    else:
        return(str(np.nan)) # If json file does not exists, we cannot search region



start = time.time()
df2['Region'] = df2[['CountryCode','City']].apply(search_region,axis=1)
end = time.time()
print(end - start)


# 390 seconds for 14k entries

19.546248197555542


In [13]:
def search_regioncode(x):
    
    filename = "CityDatabase/Cities/"+str(x['CountryCode'])+".json"

    if(os.path.exists(filename)==True):
        regions = open(filename, mode='r',encoding ="utf8")
        data = json.load(regions, encoding="utf8")

        region_query = """regions[].{region_code: code, city_names: cities[?name=='"""+str(x['City'])+"""'].name[]}"""

        path = jmespath.search(region_query,data)
        q = [x['region_code'] for x in path if (x['city_names'])]   # frigging list comprehension
        
        if(len(q)==0): # Region is not found in JSON file, hence return null
            #print("Region is not present in list")
            return(str(""))
        
        elif(len(q)==1): # Exactly 1 match is found, hence return region
            #print("Returning latitude")
            return str(q[0])
        
        elif(len(q)>1): # More than 1 regions found, hence ambiguity about the region
            #print("I am not that smart")
            return(str(""))
        data =""
        regions.close()
        
    else:
        return(str(np.nan)) # If json file does not exists, we cannot search region



start = time.time()
df2['RegionCode'] = df2[['CountryCode','City']].apply(search_regioncode,axis=1)
end = time.time()
print(end - start)


# 390 seconds for 14k entries

19.902274131774902


In [14]:
df2.head()

,City,Country,CountryCode,Latitude,Longitude,Region,RegionCode
2579,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
10188,FW-unknown,Argentina,AR,,,,
5355,Adelaide,Australia,AU,-34.8716,138.676,South Australia,05
5318,Sydney,Australia,AU,-33.8591,151.2002,New South Wales,02
11083,Melbourne,Australia,AU,-37.81,144.9644,Victoria,07


In [15]:
df2['LookupKey'] = df2['City'] + str('#') + df2['Country'] + str('#') + df2['CountryCode']

df2.head()


,City,Country,CountryCode,Latitude,Longitude,Region,RegionCode,LookupKey
2579,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07,Buenos Aires#Argentina#AR
10188,FW-unknown,Argentina,AR,,,,,FW-unknown#Argentina#AR
5355,Adelaide,Australia,AU,-34.8716,138.676,South Australia,05,Adelaide#Australia#AU
5318,Sydney,Australia,AU,-33.8591,151.2002,New South Wales,02,Sydney#Australia#AU
11083,Melbourne,Australia,AU,-37.81,144.9644,Victoria,07,Melbourne#Australia#AU


In [16]:
df2.sort_values(by=['LookupKey'], inplace=True)
df2.head()

,City,Country,CountryCode,Latitude,Longitude,Region,RegionCode,LookupKey
10057,Aarhus,Denmark,DK,56.1112,10.2099,Central Jutland,18,Aarhus#Denmark#DK
5355,Adelaide,Australia,AU,-34.8716,138.676,South Australia,05,Adelaide#Australia#AU
7912,Aguascalientes,Mexico,MX,21.8254,-102.3267,Aguascalientes,01,Aguascalientes#Mexico#MX
2209,Aix-en-Provence,France,FR,43.5312,5.4554,Provence-Alpes-Côte d'Azur,PAC,Aix-en-Provence#France#FR
7017,Alcobendas,Spain,ES,40.5458,-3.6443,Madrid,29,Alcobendas#Spain#ES


In [17]:
df2.to_csv("Output/FastLookupTable.csv", encoding="utf8", index=False)

In [18]:
df3 = pd.merge(df,df2, on='LookupKey', how ='inner')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14761 entries, 0 to 14760
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Network        14761 non-null  object
 1   City_x         14761 non-null  object
 2   Country_x      14761 non-null  object
 3   CountryCode_x  14761 non-null  object
 4   LookupKey      14761 non-null  object
 5   City_y         14761 non-null  object
 6   Country_y      14761 non-null  object
 7   CountryCode_y  14761 non-null  object
 8   Latitude       14761 non-null  object
 9   Longitude      14761 non-null  object
 10  Region         14761 non-null  object
 11  RegionCode     14761 non-null  object
dtypes: object(12)
memory usage: 1.5+ MB


In [19]:
df3.to_csv("Output/output06-gnoc_networks-merged.csv", encoding="utf8", index=False)

In [20]:
# Remove Null entries if the Region is not 

df4 = df3.copy()
df4.head()

,Network,City_x,Country_x,CountryCode_x,LookupKey,City_y,Country_y,CountryCode_y,Latitude,Longitude,Region,RegionCode
0,10.129.9.192/26,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
1,10.129.13.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
2,10.129.3.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
3,10.129.2.0/24,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
4,10.129.7.0/27,Buenos Aires,Argentina,AR,Buenos Aires#Argentina#AR,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07


In [21]:
df4.drop(columns=['LookupKey', 'City_y', 'Country_y', 'CountryCode_y'], inplace=True)
df4.head()

,Network,City_x,Country_x,CountryCode_x,Latitude,Longitude,Region,RegionCode
0,10.129.9.192/26,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
1,10.129.13.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
2,10.129.3.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
3,10.129.2.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
4,10.129.7.0/27,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07


In [22]:
df4.columns = ['NetworkIPv4_CIDR','CityName','CountryName','CountryCode','Latitude','Longitude','RegionName','RegionCode']
df4.head()

,NetworkIPv4_CIDR,CityName,CountryName,CountryCode,Latitude,Longitude,RegionName,RegionCode
0,10.129.9.192/26,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
1,10.129.13.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
2,10.129.3.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
3,10.129.2.0/24,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07
4,10.129.7.0/27,Buenos Aires,Argentina,AR,-34.6021,-58.3845,Buenos Aires F.D.,07


In [23]:
# Rearrange the Columns as per the Dynatrace Guideline

df4 = df4[['NetworkIPv4_CIDR', 'CountryName', 'CountryCode', 'RegionName','RegionCode','CityName','Latitude', 'Longitude']]
df4.tail()

,NetworkIPv4_CIDR,CountryName,CountryCode,RegionName,RegionCode,CityName,Latitude,Longitude
14756,10.61.229.0/27,Vietnam,VN,,,Ho Chi Minh,,
14757,10.61.228.192/26,Vietnam,VN,,,Ho Chi Minh,,
14758,10.61.224.248/29,Vietnam,VN,,,Ho Chi Minh,,
14759,10.61.224.32/27,Vietnam,VN,,,Ho Chi Minh,,
14760,10.61.229.32/27,Vietnam,VN,,,Ho Chi Minh,,


In [24]:
# Write to CSV file
df4.to_csv("Output/output07-gnoc_networks-dtformatted.csv", encoding="utf8", index=False)
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14761 entries, 0 to 14760
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NetworkIPv4_CIDR  14761 non-null  object
 1   CountryName       14761 non-null  object
 2   CountryCode       14761 non-null  object
 3   RegionName        14761 non-null  object
 4   RegionCode        14761 non-null  object
 5   CityName          14761 non-null  object
 6   Latitude          14761 non-null  object
 7   Longitude         14761 non-null  object
dtypes: object(8)
memory usage: 1.0+ MB


In [25]:
# Separate the rows where the regions are null


df4['RegionName'].replace('', np.nan, inplace=True)

df4_regionname_not_found = df4[df4['RegionName'].isna()]

df4_regionname_not_found.head()




,NetworkIPv4_CIDR,CountryName,CountryCode,RegionName,RegionCode,CityName,Latitude,Longitude
30,10.129.30.48/28,Argentina,AR,NaN,,FW-unknown,,
31,10.129.43.192/27,Argentina,AR,NaN,,FW-unknown,,
32,10.129.32.224/28,Argentina,AR,NaN,,FW-unknown,,
33,10.129.32.128/27,Argentina,AR,NaN,,FW-unknown,,
34,10.129.36.0/25,Argentina,AR,NaN,,FW-unknown,,


In [26]:
df4_regionname_not_found.to_csv("Output/dropped04-gnoc_networks-region-lookup-failed.csv", index=False)

print("Number of rows with region not found: "+str(len(df4_regionname_not_found)))
 
df4.dropna(subset=['RegionName'], inplace=True)

print("Number of rows after dropping missing city or country: "+str(len(df4)))

Number of rows with region not found: 3623
Number of rows after dropping missing city or country: 11138


In [27]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11138 entries, 0 to 14718
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NetworkIPv4_CIDR  11138 non-null  object
 1   CountryName       11138 non-null  object
 2   CountryCode       11138 non-null  object
 3   RegionName        11138 non-null  object
 4   RegionCode        11138 non-null  object
 5   CityName          11138 non-null  object
 6   Latitude          11138 non-null  object
 7   Longitude         11138 non-null  object
dtypes: object(8)
memory usage: 783.1+ KB


In [28]:
# FINAL OUTPUT Without IPConflict
df4.to_csv("Output/output08-gnoc_networks-final_with_ipconflict.csv", encoding="utf8", index=False)

In [29]:
df5 = df4.copy()
df5.head()

,NetworkIPv4_CIDR,CountryName,CountryCode,RegionName,RegionCode,CityName,Latitude,Longitude
0,10.129.9.192/26,Argentina,AR,Buenos Aires F.D.,07,Buenos Aires,-34.6021,-58.3845
1,10.129.13.0/24,Argentina,AR,Buenos Aires F.D.,07,Buenos Aires,-34.6021,-58.3845
2,10.129.3.0/24,Argentina,AR,Buenos Aires F.D.,07,Buenos Aires,-34.6021,-58.3845
3,10.129.2.0/24,Argentina,AR,Buenos Aires F.D.,07,Buenos Aires,-34.6021,-58.3845
4,10.129.7.0/27,Argentina,AR,Buenos Aires F.D.,07,Buenos Aires,-34.6021,-58.3845


In [30]:
# df5 = df5[['NetworkIPv4_CIDR','CountryName','RegionCode','CityName','Latitude','Longitude']]
df5 = df5[['NetworkIPv4_CIDR','CountryName','RegionName','CityName','Latitude','Longitude']]
df5.head()



,NetworkIPv4_CIDR,CountryName,RegionName,CityName,Latitude,Longitude
0,10.129.9.192/26,Argentina,Buenos Aires F.D.,Buenos Aires,-34.6021,-58.3845
1,10.129.13.0/24,Argentina,Buenos Aires F.D.,Buenos Aires,-34.6021,-58.3845
2,10.129.3.0/24,Argentina,Buenos Aires F.D.,Buenos Aires,-34.6021,-58.3845
3,10.129.2.0/24,Argentina,Buenos Aires F.D.,Buenos Aires,-34.6021,-58.3845
4,10.129.7.0/27,Argentina,Buenos Aires F.D.,Buenos Aires,-34.6021,-58.3845


In [31]:
df5.to_csv("Output/output09-gnoc_networks-final_with_ipconflict-02.csv", encoding="utf8", index=False)